### Import evaluation dataset and dependecies

Records is a list of dictionaries. Each dictionary specifies the original language of the article, metadata such as title and abstract, and a list of keywords. Each keyword is a dictionary with specified: the label, the wikidata_url, and the match. This is present even if the keyword is not in the original language of the article. 

In [1]:
from src.utils import parse_excel_file, compute_precision, compute_recall
from src.clients import OpenAIClient, AnthropicClient, GroqClient, OpenAIWebSearchClient
from src.pipelines import EntityExtractionPipeline
import os
import uuid
from datetime import datetime
import json

In [2]:
total_records = parse_excel_file("./data/Dset_Eval_KW_Alignment_Eval_def.xlsx")
records = total_records[:10]

### Definition of the client

Run the load_dotenv() for load all the env variables and run the cell with your client and 

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
model = 'openai'
client = OpenAIClient(api_key=os.getenv("OPENAI_API_KEY"), model_name="gpt-4o-mini")

In [ ]:
model = 'openai_websearch'
client = OpenAIWebSearchClient(api_key=os.getenv("OPENAI_API_KEY"), model_name="gpt-4o-search-preview")

In [ ]:
model = 'anthropic'
client = AnthropicClient(api_key=os.getenv("ANTHROPIC_API_KEY"), model_name="claude-opus-4-20250514")

In [ ]:
model = 'groq'
client = GroqClient(api_key=os.getenv("GROQ_API_KEY"), model_name="llama-3.1-8b-instant")

### 🔄 Pipeline Initialization

This cell runs the entity extraction pipeline using the input `.xlsx` file. It processes each article in the dataset and does the following for every keyword:

1. **Generate Entity Candidates**
   For each keyword, it generates up to 10 possible matching entities from Wikidata (default value set by `NUM_NAMES`).

2. **Select Relevant Entity**
   From the candidates, the model selects the best matching entity (by default, only 1 is selected).

3. **Track Progress and Save Results**
   The results are saved in a `.json` file with the original article data enriched by the LLM annotations for each keyword.

In [5]:
run_id = str(uuid.uuid4())

In [6]:
pipeline = EntityExtractionPipeline(client)

In [7]:
SAVE_EVERY = 3
MAX_WORKERS = 10
base_output_dir = './data/evaluation_output'
base_filename = f'eval_{model}_{run_id}'
results_file = os.path.join(base_output_dir, f"{base_filename}.json")
adjusted_file = os.path.join(base_output_dir, f"{base_filename}_adjusted.json")

In [8]:
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import copy

file_lock = threading.Lock()

def process_keyword(record_data):
    record_idx, kw_idx, language, title_or, abstract_or, kw_label = record_data
    
    try:
        llm_uris = pipeline.extract_entities(
            language=language,
            title=title_or,
            abstract=abstract_or,
            keywords=kw_label
        )
        return record_idx, kw_idx, llm_uris, None
    except Exception as e:
        print(f"LLM URIs cannot be computed for record {record_idx}, keyword {kw_idx}: {e}")
        return record_idx, kw_idx, [], str(e)

tasks_data = []
for record_idx, record in enumerate(records):
    for kw_idx, kw in enumerate(record['kws']):
        task_data = (
            record_idx,
            kw_idx,
            record['language'],
            record['title_eng'],
            record['abstract_eng'],
            kw['label']
        )
        tasks_data.append(task_data)

print(f"Preparati {len(tasks_data)} task da processare")

Preparati 48 task da processare


In [9]:

# Processa in parallelo
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Sottometti tutti i task con dati già copiati
    future_to_task = {
        executor.submit(process_keyword, task_data): task_data[0:2]  # record_idx, kw_idx
        for task_data in tasks_data
    }
    
    processed_count = 0
    with tqdm(total=len(tasks_data), desc="Processing keywords") as pbar:
        for future in as_completed(future_to_task):
            record_idx, kw_idx, llm_uris, error = future.result()
            
            # Aggiorna i risultati
            records[record_idx]['kws'][kw_idx]['llm_uris'] = llm_uris
            
            processed_count += 1
            pbar.update(1)
            
            # Salvataggio periodico thread-safe
            if processed_count % SAVE_EVERY == 0:
                with file_lock:
                    # print(f"Saving partial results at {processed_count} processed keywords")
                    with open(results_file, 'w', encoding='utf-8') as f:
                        json.dump(records, f, ensure_ascii=False, indent=2)

# Salvataggio finale
print("All keywords processed, saving final results.")
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(records, f, ensure_ascii=False, indent=2)

Processing keywords:   0%|          | 0/48 [00:00<?, ?it/s]

Entity: Western riding; Description: style of horseback riding which evolved from the ranching and warfare traditions; URI: http://www.wikidata.org/entity/Q1323015
Entity: Western riding; Description: ; URI: http://www.wikidata.org/entity/Q7988530

Entity: horse; Description: domesticated four-footed mammal from the equine family; URI: http://www.wikidata.org/entity/Q726
Entity: Horsens; Description: city in Horsens Municipality, Denmark; URI: http://www.wikidata.org/entity/Q190235
Entity: horse racing; Description: equestrian sport in which several horses simultaneously race against each other; URI: http://www.wikidata.org/entity/Q187916

Entity: Engineering; Description: international company located in Rome, Italy; URI: http://www.wikidata.org/entity/Q3725352
Entity: engineering; Description: applied science; URI: http://www.wikidata.org/entity/Q11023
Entity: Engineering Mechanics; Description: journal; URI: http://www.wikidata.org/entity/Q96329007
Entity: Technology; Description: 1

Processing keywords:   6%|▋         | 3/48 [00:01<00:21,  2.12it/s]

Entity: Expert; Description: Swiss company that provides support, services and solutions to retailers of consumer electronics; URI: http://www.wikidata.org/entity/Q680990
Entity: expert; Description: person with broad and profound competence in a particular field; URI: http://www.wikidata.org/entity/Q381160
Entity: expert system; Description: computer system that emulates the decision-making ability of a human expert; URI: http://www.wikidata.org/entity/Q184609
Entity: The Specialist; Description: 1994 film by Luis Llosa; URI: http://www.wikidata.org/entity/Q471423
Entity: specialist; Description: person who has completed a commercial or other vocational training, a college or a university degree; URI: http://www.wikidata.org/entity/Q1391344
Entity: specialist; Description: military rank; URI: http://www.wikidata.org/entity/Q1434096
Entity: Authority; Description: Farrar, Straus and Giroux paperback edition; URI: http://www.wikidata.org/entity/Q80550497
Entity: Authority; Description: 

Processing keywords:  12%|█▎        | 6/48 [00:03<00:19,  2.13it/s]

...

Entity: Latin inscriptions; Description: inscriptions on tapestries in the Applied Arts Collection of Milan, Italy, by Bramantino; URI: http://www.wikidata.org/entity/Q131628832
Entity: Latin inscriptions in Dante's tomb; Description: inscription in the Dante Alighieri's tomb, Ravenna, Italy; URI: http://www.wikidata.org/entity/Q131937085
Entity: Latin Inscriptions: Studies in Measurement and Making; Description: ; URI: http://www.wikidata.org/entity/Q59395786
Entity: Roman Inscriptions 1981–5; Description: ; URI: http://www.wikidata.org/entity/Q60310686
Entity: Roman Inscriptions 1986–90; Description: ; URI: http://www.wikidata.org/entity/Q60310675
Entity: Roman Inscriptions of Britain; Description: 3-volume corpus of inscriptions found in Britain from the Roman period; URI: http://www.wikidata.org/entity/Q3440860
Entity: Epigraphy; Description: sixth chapter in The Oxford Handbook of Roman Studies, 2010; URI: http://www.wikidata.org/entity/Q124353416
Entity: epigraphy; Description: s

Processing keywords:  96%|█████████▌| 46/48 [00:19<00:00,  2.31it/s]

Entity: Latin epigraphy; Description: ; URI: http://www.wikidata.org/entity/Q2352767
Entity: Latin Epigraphy: An Introduction to the Study of Latin Inscriptions. John Edwin Sandys; Description: ; URI: http://www.wikidata.org/entity/Q59735412
Entity: Latin Epigraphy, an Introduction to the Study of Latin Inscriptions. John Edwin Sandys , S. G. Campbell; Description: ; URI: http://www.wikidata.org/entity/Q59724788
Entity: Latin inscriptions; Description: inscriptions on tapestries in the Applied Arts Collection of Milan, Italy, by Bramantino; URI: http://www.wikidata.org/entity/Q131628832
Entity: Latin inscriptions in Dante's tomb; Description: inscription in the Dante Alighieri's tomb, Ravenna, Italy; URI: http://www.wikidata.org/entity/Q131937085
Entity: Latin Inscriptions: Studies in Measurement and Making; Description: ; URI: http://www.wikidata.org/entity/Q59395786
Entity: Ancient inscriptions in Somalia; Description: Undeciphered writing systems in Somalia; URI: http://www.wikidata

Processing keywords: 100%|██████████| 48/48 [00:28<00:00,  1.67it/s]

All keywords processed, saving final results.


The following cell of code contains some required adjustments to the evaluation output for the results analysis. These adjustments mainly involve string parsing. Running this cell is required to obtain the statistics. 

In [21]:
new_records = []

with open(results_file, 'r', encoding='utf-8') as f:
    records = json.load(f)


for record in records:
    new_record = record
    for i, kw in enumerate(record['kws']):
        if len(kw['llm_uris']) == 3 and kw['llm_uris'][1] == "wikidata":
            new_record['kws'][i]['llm_uris'] = ['.'.join(kw['llm_uris'])]
    new_records.append(new_record)

with open(adjusted_file, 'w', encoding='utf-8') as f:
    json.dump(new_records, f, ensure_ascii=False, indent=2)
    

### Evaluation

The following two cells can be used to obtain the statistics. As can be seen, the recall, precision, and the F1 scores are printed. They are printed for all the examples, for language (that is, we distinguish the results based on the original language of the article of the keyword), and by match type (where for match type we consider how match the Wikidata URL matches the actual keyword, this could be 'e' - exact match -, or 'r' - related match).

As can be seen in the example below, the code prints a report with the results. 

In [22]:
with open(adjusted_file, 'r', encoding='utf-8') as f:
    records = json.load(f)

languages = set(record['language'] for record in records)

scores_llm = {
    "Total": {
        "recall": {"Sum": 0, "Size": 0},
        "precision": {"Sum": 0, "Size": 0},
        "f1": {"Sum": 0, "Size": 0}
    },
    "Per_match_type": {
        "e": {
            "recall": {"Sum": 0, "Size": 0},
            "precision": {"Sum": 0, "Size": 0},
            "f1": {"Sum": 0, "Size": 0}
        },
        "r": {
            "recall": {"Sum": 0, "Size": 0},
            "precision": {"Sum": 0, "Size": 0},
            "f1": {"Sum": 0, "Size": 0}
        }
    },
    "Per_language": {
        language: {
            "recall": {"Sum": 0, "Size": 0},
            "precision": {"Sum": 0, "Size": 0},
            "f1": {"Sum": 0, "Size": 0}
        } for language in languages
    }
}

for record in records:
    for i, kw in enumerate(record['kws']):
        if kw['match'] in ("e", "r"):
            correct_uris = [
                url.replace("https", "http").replace("/wiki/", "/entity/")
                for url in kw['wikidata_url']
            ]
            retrieved_uris_llm = kw['llm_uris']
            recall_llm = compute_recall(correct_uris, retrieved_uris_llm)
            precision_llm = compute_precision(correct_uris, retrieved_uris_llm)
            
            # Calcolo del punteggio F1 con controllo per divisione per zero
            if (precision_llm + recall_llm) > 0:
                f1_llm = 2 * precision_llm * recall_llm / (precision_llm + recall_llm)
            else:
                f1_llm = 0

            # Aggiornamento dei punteggi totali
            scores_llm["Total"]["recall"]["Sum"] += recall_llm
            scores_llm["Total"]["recall"]["Size"] += 1
            scores_llm["Total"]["precision"]["Sum"] += precision_llm
            scores_llm["Total"]["precision"]["Size"] += 1
            scores_llm["Total"]["f1"]["Sum"] += f1_llm
            scores_llm["Total"]["f1"]["Size"] += 1

            # Aggiornamento dei punteggi per tipo di match
            scores_llm["Per_match_type"][kw['match']]["recall"]["Sum"] += recall_llm
            scores_llm["Per_match_type"][kw['match']]["recall"]["Size"] += 1
            scores_llm["Per_match_type"][kw['match']]["precision"]["Sum"] += precision_llm
            scores_llm["Per_match_type"][kw['match']]["precision"]["Size"] += 1
            scores_llm["Per_match_type"][kw['match']]["f1"]["Sum"] += f1_llm
            scores_llm["Per_match_type"][kw['match']]["f1"]["Size"] += 1

            # Aggiornamento dei punteggi per lingua
            scores_llm["Per_language"][record['language']]["recall"]["Sum"] += recall_llm
            scores_llm["Per_language"][record['language']]["recall"]["Size"] += 1
            scores_llm["Per_language"][record['language']]["precision"]["Sum"] += precision_llm
            scores_llm["Per_language"][record['language']]["precision"]["Size"] += 1
            scores_llm["Per_language"][record['language']]["f1"]["Sum"] += f1_llm
            scores_llm["Per_language"][record['language']]["f1"]["Size"] += 1


In [23]:
import json

def compute_mean_metrics(stats_dict):
    """
    Dato un dizionario con la struttura:
    {
      'Total': {
        'recall': {'Sum': x, 'Size': y},
        'precision': {'Sum': x2, 'Size': y2},
        'f1': {'Sum': x3, 'Size': y3}
      },
      # 'Per_match_type': {
        'e': {
          'recall': {'Sum': x, 'Size': y},
          'precision': {'Sum': x2, 'Size': y2},
          'f1': {'Sum': x3, 'Size': y3}
        },
        ...
      },
      'Per_language': {
        'fr': {
          'recall': {'Sum': x, 'Size': y},
          'precision': {'Sum': x2, 'Size': y2},
          'f1': {'Sum': x3, 'Size': y3}
        },
        ...
      }
    }

    Restituisce un dizionario con i valori medi di recall, precision e f1.
    """
    mean_dict = {
        'Total': {},
        'Per_match_type': {},
        'Per_language': {}
    }
    
    # --- 1) TOTAL ---
    if 'Total' in stats_dict:
        total_recall_sum = stats_dict['Total']['recall']['Sum']
        total_recall_size = stats_dict['Total']['recall']['Size']
        total_precision_sum = stats_dict['Total']['precision']['Sum']
        total_precision_size = stats_dict['Total']['precision']['Size']
        
        mean_dict['Total']['recall'] = total_recall_sum / total_recall_size if total_recall_size != 0 else 0
        mean_dict['Total']['precision'] = total_precision_sum / total_precision_size if total_precision_size != 0 else 0
        
        if 'f1' in stats_dict['Total']:
            total_f1_sum = stats_dict['Total']['f1']['Sum']
            total_f1_size = stats_dict['Total']['f1']['Size']
            mean_dict['Total']['f1'] = total_f1_sum / total_f1_size if total_f1_size != 0 else 0

    # --- 2) PER MATCH TYPE ---
    if 'Per_match_type' in stats_dict:
        for match_type, metrics in stats_dict['Per_match_type'].items():
            recall_sum = metrics['recall']['Sum']
            recall_size = metrics['recall']['Size']
            precision_sum = metrics['precision']['Sum']
            precision_size = metrics['precision']['Size']
            
            mean_rec = recall_sum / recall_size if recall_size != 0 else 0
            mean_prec = precision_sum / precision_size if precision_size != 0 else 0
            
            mean_dict['Per_match_type'][match_type] = {
                'recall': mean_rec,
                'precision': mean_prec
            }
            if 'f1' in metrics:
                f1_sum = metrics['f1']['Sum']
                f1_size = metrics['f1']['Size']
                mean_dict['Per_match_type'][match_type]['f1'] = f1_sum / f1_size if f1_size != 0 else 0

    # --- 3) PER LANGUAGE ---
    if 'Per_language' in stats_dict:
        for lang, metrics in stats_dict['Per_language'].items():
            recall_sum = metrics['recall']['Sum']
            recall_size = metrics['recall']['Size']
            precision_sum = metrics['precision']['Sum']
            precision_size = metrics['precision']['Size']
            
            mean_rec = recall_sum / recall_size if recall_size != 0 else 0
            mean_prec = precision_sum / precision_size if precision_size != 0 else 0
            
            mean_dict['Per_language'][lang] = {
                'recall': mean_rec,
                'precision': mean_prec
            }
            if 'f1' in metrics:
                f1_sum = metrics['f1']['Sum']
                f1_size = metrics['f1']['Size']
                mean_dict['Per_language'][lang]['f1'] = f1_sum / f1_size if f1_size != 0 else 0
                
    return mean_dict


def print_system_report(system_dict):
    """
    Stampa un report leggibile dei risultati del sistema (LLM) per Total,
    Per_match_type e Per_language includendo F1.
    """
    system_means = compute_mean_metrics(system_dict)
    
    print("======== SYSTEM METRICS REPORT ========")
    
    # --- 1) TOTAL ---
    print("\n--- TOTAL ---")
    total = system_means.get('Total', {})
    if total:
        print(f"LLM Recall:    {total.get('recall', 0):.4f}")
        print(f"LLM Precision: {total.get('precision', 0):.4f}")
        print(f"LLM F1:        {total.get('f1', 0):.4f}")
    else:
        print("No total data found.")
    
    # --- 2) PER MATCH TYPE ---
    print("\n--- PER MATCH TYPE ---")
    per_match_data = system_means.get('Per_match_type', {})
    for mtype, vals in per_match_data.items():
        print(f"\nMatch Type: {mtype}")
        print(f"  LLM Recall:    {vals.get('recall', 0):.4f}")
        print(f"  LLM Precision: {vals.get('precision', 0):.4f}")
        print(f"  LLM F1:        {vals.get('f1', 0):.4f}")
    
    # --- 3) PER LANGUAGE ---
    print("\n--- PER LANGUAGE ---")
    per_lang_data = system_means.get('Per_language', {})
    # Ordinamento convertendo le chiavi in stringa per evitare errori se sono di tipo misto
    for lang, vals in sorted(per_lang_data.items(), key=lambda x: str(x[0])):
        print(f"\nLanguage: {lang}")
        print(f"  LLM Recall:    {vals.get('recall', 0):.4f}")
        print(f"  LLM Precision: {vals.get('precision', 0):.4f}")
        print(f"  LLM F1:        {vals.get('f1', 0):.4f}")


# Esempio di utilizzo:
# Supponiamo che 'scores_llm' contenga i risultati del sistema (LLM)
# Ad esempio, scores_llm è stato popolato precedentemente nel codice.
print_system_report(scores_llm)

======== SYSTEM METRICS REPORT ========

--- TOTAL ---
LLM Recall:    0.5590
LLM Precision: 0.5833
LLM F1:        0.5660

--- PER MATCH TYPE ---

Match Type: e
  LLM Recall:    0.6301
  LLM Precision: 0.6585
  LLM F1:        0.6382

Match Type: r
  LLM Recall:    0.1429
  LLM Precision: 0.1429
  LLM F1:        0.1429

--- PER LANGUAGE ---

Language: ar
  LLM Recall:    1.0000
  LLM Precision: 1.0000
  LLM F1:        1.0000

Language: el
  LLM Recall:    0.4286
  LLM Precision: 0.4286
  LLM F1:        0.4286

Language: es
  LLM Recall:    0.7500
  LLM Precision: 0.7500
  LLM F1:        0.7500

Language: fi
  LLM Recall:    0.5556
  LLM Precision: 0.6667
  LLM F1:        0.5833

Language: it
  LLM Recall:    0.5556
  LLM Precision: 0.5556
  LLM F1:        0.5556

Language: pl
  LLM Recall:    0.5000
  LLM Precision: 0.5714
  LLM F1:        0.5238

Language: sl
  LLM Recall:    0.3333
  LLM Precision: 0.3333
  LLM F1:        0.3333

Language: sv
  LLM Recall:    0.5000
  LLM Precision: 0.